# // This script merges the cbi outputs from generate_gf_cbi


In [1]:
import os
from functools import reduce

import ee

from find_set_root import find_set_project_root
PROJECT_ROOT = find_set_project_root()
print(f"Project root found at: {PROJECT_ROOT}")
import utils.gee_functions as ugeef

ee.Initialize()

# USER-SET PARAMETERS
PROJECT_DIR = "projects/ee-tymc5571-goodfire/assets/"
VERSION = "welty_wildfire"


Project root found at: C:\Users\tymc5571\dev\a-number-on-good-fire


In [2]:
# Get assets in the specified project directory
assets = ugeef.list_assets(PROJECT_DIR)
# print(f"✅ Found {len(assets)} assets:")
# print(assets)
# print(type(assets))

# Get assets of interest (those containing the version and "cbi_bc_batch_")
cbi_batch_assets = [s for s in assets if VERSION in s and "cbi_bc_batch" in s]
sorted_cbi_batch_assets = sorted(cbi_batch_assets)
print(sorted_cbi_batch_assets)

['projects/ee-tymc5571-goodfire/assets/welty_wildfire_cbi_bc_batch_1985_1989', 'projects/ee-tymc5571-goodfire/assets/welty_wildfire_cbi_bc_batch_1990_1994', 'projects/ee-tymc5571-goodfire/assets/welty_wildfire_cbi_bc_batch_1995_1999', 'projects/ee-tymc5571-goodfire/assets/welty_wildfire_cbi_bc_batch_2000_2004', 'projects/ee-tymc5571-goodfire/assets/welty_wildfire_cbi_bc_batch_2005_2009', 'projects/ee-tymc5571-goodfire/assets/welty_wildfire_cbi_bc_batch_2010_2014', 'projects/ee-tymc5571-goodfire/assets/welty_wildfire_cbi_bc_batch_2015_2019', 'projects/ee-tymc5571-goodfire/assets/welty_wildfire_cbi_bc_batch_2020_2020']


In [4]:
print(sorted_cbi_batch_assets)
images = [ee.Image(asset_id) for asset_id in sorted_cbi_batch_assets] #Create ee.Images from assets
all_cbi = reduce(lambda img1, img2: img1.addBands(img2), images) #Merge
print(all_cbi.bandNames().getInfo())

# Get the earliest and latest year to use in naming the output
band_names = all_cbi.bandNames().getInfo()  # e.g., ['cbi_year_1985', 'cbi_year_1986', ...]
years = [int(name.replace('cbi_year_', '')) for name in band_names if name.startswith('cbi_year_')]
EARLIEST_YR = str(min(years))
LATEST_YR = str(max(years))
print(EARLIEST_YR, LATEST_YR)

['projects/ee-tymc5571-goodfire/assets/welty_wildfire_cbi_bc_batch_1985_1989', 'projects/ee-tymc5571-goodfire/assets/welty_wildfire_cbi_bc_batch_1990_1994', 'projects/ee-tymc5571-goodfire/assets/welty_wildfire_cbi_bc_batch_1995_1999', 'projects/ee-tymc5571-goodfire/assets/welty_wildfire_cbi_bc_batch_2000_2004', 'projects/ee-tymc5571-goodfire/assets/welty_wildfire_cbi_bc_batch_2005_2009', 'projects/ee-tymc5571-goodfire/assets/welty_wildfire_cbi_bc_batch_2010_2014', 'projects/ee-tymc5571-goodfire/assets/welty_wildfire_cbi_bc_batch_2015_2019', 'projects/ee-tymc5571-goodfire/assets/welty_wildfire_cbi_bc_batch_2020_2020']
['cbi_year_1985', 'cbi_year_1986', 'cbi_year_1987', 'cbi_year_1988', 'cbi_year_1989', 'cbi_year_1990', 'cbi_year_1991', 'cbi_year_1992', 'cbi_year_1993', 'cbi_year_1994', 'cbi_year_1995', 'cbi_year_1996', 'cbi_year_1997', 'cbi_year_1998', 'cbi_year_1999', 'cbi_year_2000', 'cbi_year_2001', 'cbi_year_2002', 'cbi_year_2003', 'cbi_year_2004', 'cbi_year_2005', 'cbi_year_2006', 

In [5]:

# Define the export region
states = ee.FeatureCollection("TIGER/2018/States")
study_area_west = states.filter(
    ee.Filter.inList('NAME', [
        'Washington', 'Oregon', 'California', 'Idaho', 'Montana', 'Nevada',
        'Colorado', 'Wyoming', 'New Mexico', 'Arizona', 'Utah'
    ])
)

output_region = study_area_west.geometry()

# Export to Earth Engine Asset
task = ee.batch.Export.image.toAsset(
    image=all_cbi.toFloat(),
    description= VERSION + "_cbi_bc_" + EARLIEST_YR + "_" + LATEST_YR,
    assetId="projects/ee-tymc5571-goodfire/assets/" + VERSION + "_cbi_bc_" + EARLIEST_YR + "_" + LATEST_YR,
    scale=30,
    crs="EPSG:4326",
    region=output_region,
    maxPixels=1e13
)

# Start and monitor
task.start()
# print("🚀 Export task started")

# # Monitor the task status
# while task.active():
#     print("⏳ Waiting for export to complete...")
#     time.sleep(300)  # Wait 30 seconds before checking again

# # Final status check
# status = task.status()
# if status['state'] == 'COMPLETED':
#     print("✅ Export completed successfully!")
# else:
#     print(f"❌ Export failed with status: {status['state']}")
#     print("Details:", status)